In [21]:
from kliff.dataset import Dataset
from kliff.ml.opt import OptimizerTorch
from kliff.ml.libdescriptor import Descriptor
from kliff.ml.training_wheels import TrainingWheels
from kliff.ml.libdescriptor.descriptor import get_default_bispectrum
import numpy as np
import torch
from torch.nn import Sequential, Linear, ReLU, Tanh
torch.set_default_tensor_type(torch.DoubleTensor)

In [22]:
dataset = Dataset("./")

2023-03-23 11:16:55.889 | INFO     | kliff.dataset.dataset:_read:628 - 4 configurations read from /home/amit/Documents/Presentations/Demos/SciML/example


In [23]:
hyperparameters = get_default_bispectrum()
hyperparameters

OrderedDict([('jmax', 4),
             ('rfac0', 0.99363),
             ('diagonalstyle', 3),
             ('rmin0', 0),
             ('switch_flag', 1),
             ('bzero_flag', 0),
             ('use_shared_array', False),
             ('weights', None)])

In [24]:
desc = Descriptor(species=["Si"], cutoff=3.77, descriptor="Bispectrum",hyperparameters=hyperparameters)
model = Sequential(Linear(desc.width, 10), ReLU(), Linear(10, 10), Tanh(), Linear(10, 1))

In [26]:
print(desc.forward(dataset[0]))

[[11.21279788  1.32144451  0.71864351 ...  6.08358475  9.12367976
   4.64763422]
 [11.71480386  1.38341366  0.71840706 ...  5.86675604  9.25307807
   4.03739021]
 [11.56239684  1.36223379  0.71805468 ...  5.91457133  9.18903281
   4.21462028]
 ...
 [11.70718464  1.38402086  0.7198934  ...  5.89704364  9.26167241
   4.08735836]
 [11.44231973  1.35056181  0.71991062 ...  6.00569278  9.17827144
   4.41222088]
 [11.45862475  1.35355995  0.71916522 ...  5.99261215  9.19334183
   4.37218574]]


### Compute energy and forces

In [27]:
# get descriptor
representation = torch.tensor(desc.forward(dataset[0]))
representation.requires_grad_(True)

# energy
atomwise_energy = model(representation)
total_energy = atomwise_energy.sum()

# dE/d desc
total_energy.backward()
dE_ddesc = representation.grad.detach().numpy()

# dE/dr = dE/d desc x J_bispectrum 
forces = desc.backward(dataset[0], dE_ddesc)
print(forces)

[[ 0.01194377 -0.02445296  0.0428753 ]
 [-0.0161087   0.0510593  -0.0248493 ]
 [ 0.04212561  0.00338546 -0.06085004]
 [-0.0186182   0.02514027 -0.02311917]
 [ 0.05752531  0.02175386  0.00469831]
 [ 0.00477205 -0.02662726 -0.01296752]
 [-0.00514395  0.03771436 -0.03107111]
 [-0.03357846  0.01244308 -0.02685095]
 [-0.08091077  0.06516367  0.02695972]
 [-0.00395366 -0.00677185 -0.0199847 ]
 [ 0.01001026 -0.00648298 -0.04340275]
 [ 0.02339664  0.01723912  0.01371598]
 [-0.02954064 -0.01868695 -0.05451479]
 [ 0.03268467  0.05953101  0.04056934]
 [ 0.00071592 -0.01955118  0.02578998]
 [-0.01351203  0.03445555  0.03927001]
 [-0.00353726  0.01427137 -0.00057532]
 [-0.02892285  0.0285444  -0.0242593 ]
 [ 0.05623262  0.01303339 -0.04340668]
 [ 0.0093282   0.04327132  0.06035917]
 [-0.02977223  0.02348511  0.04300479]
 [ 0.01078711 -0.04395722  0.00103963]
 [ 0.01646246 -0.04029043  0.00950209]
 [-0.01252855  0.01645009 -0.04571689]
 [-0.0504034   0.02324288  0.01370309]
 [-0.0501157  -0.06496425

### Using TrainingWheels

In [28]:
model_tw = TrainingWheels.init_descriptor(model=model, cutoff=3.77, descriptor_kind="Bispectrum", species=["Si"], 
                               hyperparams=hyperparameters, model_returns_forces=False)

In [29]:
model_tw(dataset[0])

{'energy': tensor(-67.1918, grad_fn=<SumBackward0>),
 'forces': tensor([[-0.0119,  0.0245, -0.0429],
         [ 0.0161, -0.0511,  0.0248],
         [-0.0421, -0.0034,  0.0609],
         [ 0.0186, -0.0251,  0.0231],
         [-0.0575, -0.0218, -0.0047],
         [-0.0048,  0.0266,  0.0130],
         [ 0.0051, -0.0377,  0.0311],
         [ 0.0336, -0.0124,  0.0269],
         [ 0.0809, -0.0652, -0.0270],
         [ 0.0040,  0.0068,  0.0200],
         [-0.0100,  0.0065,  0.0434],
         [-0.0234, -0.0172, -0.0137],
         [ 0.0295,  0.0187,  0.0545],
         [-0.0327, -0.0595, -0.0406],
         [-0.0007,  0.0196, -0.0258],
         [ 0.0135, -0.0345, -0.0393],
         [ 0.0035, -0.0143,  0.0006],
         [ 0.0289, -0.0285,  0.0243],
         [-0.0562, -0.0130,  0.0434],
         [-0.0093, -0.0433, -0.0604],
         [ 0.0298, -0.0235, -0.0430],
         [-0.0108,  0.0440, -0.0010],
         [-0.0165,  0.0403, -0.0095],
         [ 0.0125, -0.0165,  0.0457],
         [ 0.0504, -0.023

### Train model

In [30]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)
opt = OptimizerTorch(model_tw, dataset, optimizer=optimizer, epochs=5)

In [31]:
_ = opt.minimize()

2023-03-23 11:19:30.417 | INFO     | kliff.ml.opt:minimize:234 - Starting with method Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    eps: 1e-08
    foreach: None
    lr: 0.01
    maximize: False
    weight_decay: 0
)


In [72]:
model_tw.save_kim_model("ExampleModel")